# 🏀 Can We Predict a Winner?
## Machine Learning on Victorian Basketball

We've explored **47,000+ game results** from Basketball Victoria's PlayHQ platform. Now the big question: **can a machine learning model predict who wins?**

In this notebook we'll:
1. **Engineer features** from team and player statistics
2. **Build classifiers** (Random Forest & XGBoost) to predict game outcomes
3. **Predict score margins** with regression models
4. **Measure player impact** with a plus/minus proxy
5. **Simulate a season** using our trained model
6. **Compare all models** and discuss what works — and what doesn't

Let's find out what it takes to win in Victorian basketball. 🏆

In [1]:
import sqlite3
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                             roc_auc_score, roc_curve, mean_squared_error, r2_score)
from xgboost import XGBClassifier, XGBRegressor

pd.set_option('display.max_columns', 50)

# Connect
conn = sqlite3.connect('../data/playhq.db')

# Load core tables
games = pd.read_sql("SELECT * FROM games WHERE status='FINAL' AND home_score IS NOT NULL AND away_score IS NOT NULL", conn)
player_stats = pd.read_sql("SELECT * FROM player_stats", conn)
teams = pd.read_sql("SELECT * FROM teams", conn)
grades = pd.read_sql("SELECT * FROM grades", conn)
rounds = pd.read_sql("SELECT * FROM rounds", conn)
seasons = pd.read_sql("SELECT * FROM seasons", conn)

print(f"Games with final scores: {len(games):,}")
print(f"Player stat records: {len(player_stats):,}")
print(f"Teams: {len(teams):,}")

Games with final scores: 42,632
Player stat records: 160,110
Teams: 2,231


## 📐 Feature Engineering

For each game, we need to compute **team-level features** that a model can learn from. We'll build:

| Feature | Description |
|---------|-------------|
| `avg_ppg` | Team's average points per game (season to date) |
| `avg_3pt_pct` | Proportion of team's points from 3-pointers |
| `avg_fouls_pg` | Average fouls per game |
| `win_pct` | Team's win percentage coming into the game |
| `win_streak` | Current consecutive wins (negative = losses) |
| `h2h_win_pct` | Historical head-to-head win rate vs this opponent |
| `is_home` | Home court indicator |

All features are computed as **rolling stats up to but not including the current game** — no data leakage!

In [2]:
# Sort games chronologically within each grade
games['date'] = pd.to_datetime(games['date'])
games = games.sort_values(['grade_id', 'date', 'time']).reset_index(drop=True)

# Compute per-team season stats from player_stats table
# player_stats gives us totals per player per grade
team_grade_stats = player_stats.groupby(['grade_id', 'team_name']).agg(
    total_points=('total_points', 'sum'),
    total_3pt=('three_point', 'sum'),
    total_fouls=('total_fouls', 'sum'),
    total_games=('games_played', 'sum'),
    num_players=('id', 'count')
).reset_index()

team_grade_stats['avg_ppg_player'] = team_grade_stats['total_points'] / team_grade_stats['num_players'].clip(1)
team_grade_stats['three_pt_ratio'] = team_grade_stats['total_3pt'] / team_grade_stats['total_points'].clip(1)
team_grade_stats['avg_fouls_player'] = team_grade_stats['total_fouls'] / team_grade_stats['num_players'].clip(1)

print(f"Team-grade stat records: {len(team_grade_stats):,}")
team_grade_stats.head()

Team-grade stat records: 21,098


,grade_id,team_name,total_points,total_3pt,total_fouls,total_games,num_players,avg_ppg_player,three_pt_ratio,avg_fouls_player
0,008c1767,Balwyn U17 Boys 01,201,6,62,38,8,25.125,0.029851,7.750000
1,008c1767,Balwyn U17 Boys 02,100,18,26,22,8,12.500,0.180000,3.250000
2,008c1767,Balwyn U17 Boys 03,117,4,25,13,6,19.500,0.034188,4.166667
3,008c1767,Balwyn U17 Boys 05,168,13,35,29,8,21.000,0.077381,4.375000
4,008c1767,Balwyn U17 Boys 06,90,2,16,16,8,11.250,0.022222,2.000000


In [3]:
# Build rolling features game-by-game
# We'll track each team's running stats within a grade

def build_game_features(games_df):
    """Build features for each game using only prior information."""
    
    # Track running stats per team per grade
    team_stats = {}  # (grade_id, team_id) -> {wins, losses, streak, points_for, points_against, games}
    h2h_stats = {}   # (grade_id, team_a, team_b) -> {a_wins, b_wins}
    
    rows = []
    
    for _, g in games_df.iterrows():
        gid = g['grade_id']
        ht, at = g['home_team_id'], g['away_team_id']
        hs, as_ = g['home_score'], g['away_score']
        
        if pd.isna(hs) or pd.isna(as_) or ht is None or at is None:
            continue
            
        hs, as_ = int(hs), int(as_)
        
        # Get current stats (before this game)
        h_key = (gid, ht)
        a_key = (gid, at)
        
        h_st = team_stats.get(h_key, {'wins': 0, 'losses': 0, 'streak': 0, 'pf': 0, 'pa': 0, 'gp': 0})
        a_st = team_stats.get(a_key, {'wins': 0, 'losses': 0, 'streak': 0, 'pf': 0, 'pa': 0, 'gp': 0})
        
        # H2H
        h2h_key = (gid, ht, at)
        h2h_rev = (gid, at, ht)
        h2h = h2h_stats.get(h2h_key, {'wins': 0, 'losses': 0})
        
        # Compute features
        h_gp = max(h_st['gp'], 1)
        a_gp = max(a_st['gp'], 1)
        h2h_total = max(h2h['wins'] + h2h['losses'], 1)
        
        row = {
            'game_id': g['id'],
            'grade_id': gid,
            'date': g['date'],
            'home_team_id': ht,
            'away_team_id': at,
            'home_score': hs,
            'away_score': as_,
            # Home team features
            'h_win_pct': h_st['wins'] / max(h_st['wins'] + h_st['losses'], 1),
            'h_avg_ppg': h_st['pf'] / h_gp,
            'h_avg_papg': h_st['pa'] / h_gp,
            'h_streak': h_st['streak'],
            'h_games_played': h_st['gp'],
            # Away team features
            'a_win_pct': a_st['wins'] / max(a_st['wins'] + a_st['losses'], 1),
            'a_avg_ppg': a_st['pf'] / a_gp,
            'a_avg_papg': a_st['pa'] / a_gp,
            'a_streak': a_st['streak'],
            'a_games_played': a_st['gp'],
            # Differential features
            'win_pct_diff': h_st['wins'] / max(h_st['wins'] + h_st['losses'], 1) - a_st['wins'] / max(a_st['wins'] + a_st['losses'], 1),
            'ppg_diff': h_st['pf'] / h_gp - a_st['pf'] / a_gp,
            # H2H
            'h2h_home_win_pct': h2h['wins'] / h2h_total,
            # Target
            'home_win': 1 if hs > as_ else 0,
            'margin': hs - as_,
        }
        rows.append(row)
        
        # Update stats AFTER recording features
        home_won = hs > as_
        
        # Update home team
        if home_won:
            h_st['wins'] += 1
            h_st['streak'] = max(h_st['streak'], 0) + 1
            a_st['losses'] += 1
            a_st['streak'] = min(a_st['streak'], 0) - 1
        else:
            h_st['losses'] += 1
            h_st['streak'] = min(h_st['streak'], 0) - 1
            a_st['wins'] += 1
            a_st['streak'] = max(a_st['streak'], 0) + 1
            
        h_st['pf'] += hs; h_st['pa'] += as_; h_st['gp'] += 1
        a_st['pf'] += as_; a_st['pa'] += hs; a_st['gp'] += 1
        
        team_stats[h_key] = h_st
        team_stats[a_key] = a_st
        
        # Update H2H
        if home_won:
            h2h['wins'] += 1
        else:
            h2h['losses'] += 1
        h2h_stats[h2h_key] = h2h
        h2h_stats[h2h_rev] = {'wins': h2h['losses'], 'losses': h2h['wins']}
    
    return pd.DataFrame(rows)

feat_df = build_game_features(games)
print(f"Feature matrix: {feat_df.shape[0]:,} games × {feat_df.shape[1]} columns")

# Drop games where teams haven't played enough (cold start)
feat_df = feat_df[(feat_df['h_games_played'] >= 2) & (feat_df['a_games_played'] >= 2)].copy()
print(f"After filtering cold-start games: {feat_df.shape[0]:,}")
feat_df.head()

Feature matrix: 42,632 games × 22 columns
After filtering cold-start games: 27,707


,game_id,grade_id,date,home_team_id,away_team_id,home_score,away_score,h_win_pct,h_avg_ppg,h_avg_papg,h_streak,h_games_played,a_win_pct,a_avg_ppg,a_avg_papg,a_streak,a_games_played,win_pct_diff,ppg_diff,h2h_home_win_pct,home_win,margin
8,ada3ee39,00148ccf,2022-02-19,d961b5bf,1366d056,17,16,0.0,9.500000,16.000000,-2,2,0.000000,9.500000,25.5,-2,2,0.000000,0.0,0.0,1,1
9,63009819,00148ccf,2022-02-19,d5a99f70,bcf42772,12,16,1.0,29.500000,10.000000,2,2,0.500000,10.000000,15.5,-1,2,0.500000,19.5,0.0,0,-4
10,4f3d038f,00148ccf,2022-02-19,ae01defc,10a16757,16,33,0.5,14.500000,14.000000,1,2,1.000000,10.000000,9.0,2,2,-0.500000,4.5,0.0,0,-17
11,626635af,00148ccf,2022-02-19,5a350c7c,99649879,19,10,1.0,16.500000,4.000000,2,2,0.000000,8.000000,13.5,-2,2,1.000000,8.5,0.0,1,9
12,f192ca0f,00148ccf,2022-02-26,99649879,d5a99f70,1,18,0.0,8.666667,15.333333,-3,3,0.666667,23.666667,12.0,-1,3,-0.666667,-15.0,0.0,0,-17


In [4]:
# Merge in team-level 3PT and foul data from player_stats
# Map team_id -> team_name for lookup
team_name_map = dict(zip(teams['id'], teams['name']))

# Create grade-level aggregated stats
ps_agg = player_stats.groupby(['grade_id', 'team_name']).agg(
    tot_pts=('total_points', 'sum'),
    tot_3pt=('three_point', 'sum'),
    tot_fouls=('total_fouls', 'sum'),
    tot_gp=('games_played', 'sum'),
    n_players=('id', 'count')
).reset_index()

ps_agg['three_pt_pct'] = ps_agg['tot_3pt'] * 3 / ps_agg['tot_pts'].clip(1)
ps_agg['fouls_per_game'] = ps_agg['tot_fouls'] / ps_agg['tot_gp'].clip(1)

# Build lookup: (grade_id, team_name) -> stats
stat_lookup = {}
for _, r in ps_agg.iterrows():
    stat_lookup[(r['grade_id'], r['team_name'])] = {
        'three_pt_pct': r['three_pt_pct'],
        'fouls_per_game': r['fouls_per_game']
    }

# Add to feature df
def get_stat(grade_id, team_id, stat):
    name = team_name_map.get(team_id, '')
    return stat_lookup.get((grade_id, name), {}).get(stat, 0)

feat_df['h_3pt_pct'] = feat_df.apply(lambda r: get_stat(r['grade_id'], r['home_team_id'], 'three_pt_pct'), axis=1)
feat_df['a_3pt_pct'] = feat_df.apply(lambda r: get_stat(r['grade_id'], r['away_team_id'], 'three_pt_pct'), axis=1)
feat_df['h_fouls_pg'] = feat_df.apply(lambda r: get_stat(r['grade_id'], r['home_team_id'], 'fouls_per_game'), axis=1)
feat_df['a_fouls_pg'] = feat_df.apply(lambda r: get_stat(r['grade_id'], r['away_team_id'], 'fouls_per_game'), axis=1)

feat_df['three_pt_diff'] = feat_df['h_3pt_pct'] - feat_df['a_3pt_pct']
feat_df['fouls_diff'] = feat_df['h_fouls_pg'] - feat_df['a_fouls_pg']

print("Features added. Final shape:", feat_df.shape)

Features added. Final shape: (27707, 28)


In [5]:
# Quick look at feature distributions
fig = make_subplots(rows=2, cols=3, subplot_titles=[
    'Win % Differential', 'PPG Differential', 'Win Streak (Home)',
    'H2H Win %', '3PT % Differential', 'Score Margin'
])

data = [
    (feat_df['win_pct_diff'], 1, 1),
    (feat_df['ppg_diff'], 1, 2),
    (feat_df['h_streak'], 1, 3),
    (feat_df['h2h_home_win_pct'], 2, 1),
    (feat_df['three_pt_diff'], 2, 2),
    (feat_df['margin'], 2, 3),
]

for d, r, c in data:
    fig.add_trace(go.Histogram(x=d, nbinsx=50, marker_color='#1f77b4', opacity=0.7), row=r, col=c)

fig.update_layout(title='📊 Feature Distributions', showlegend=False, height=500,
                  template='plotly_white')
fig.show()

## 🎯 Game Outcome Prediction

Time to build our classifiers. We'll use:
- **Random Forest** — robust ensemble of decision trees
- **XGBoost** — gradient-boosted trees, often state-of-the-art for tabular data

We use a **temporal split**: earlier games for training, later games for testing. This is more realistic than a random split since we're predicting future games.

In [6]:
# Define feature columns
feature_cols = [
    'h_win_pct', 'h_avg_ppg', 'h_avg_papg', 'h_streak', 'h_games_played',
    'a_win_pct', 'a_avg_ppg', 'a_avg_papg', 'a_streak', 'a_games_played',
    'win_pct_diff', 'ppg_diff', 'h2h_home_win_pct',
    'h_3pt_pct', 'a_3pt_pct', 'h_fouls_pg', 'a_fouls_pg',
    'three_pt_diff', 'fouls_diff'
]

X = feat_df[feature_cols].fillna(0)
y = feat_df['home_win']

# Temporal split — use date
split_date = feat_df['date'].quantile(0.8)
train_mask = feat_df['date'] <= split_date
test_mask = feat_df['date'] > split_date

X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y[train_mask], y[test_mask]

print(f"Training set: {len(X_train):,} games (up to {split_date.date()})")
print(f"Test set:     {len(X_test):,} games (after {split_date.date()})")
print(f"\nHome win rate — Train: {y_train.mean():.1%} | Test: {y_test.mean():.1%}")

Training set: 22,668 games (up to 2025-10-25)
Test set:     5,039 games (after 2025-10-25)

Home win rate — Train: 50.8% | Test: 49.7%


In [7]:
# Train Random Forest
rf = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_leaf=20, 
                             random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_prob = rf.predict_proba(X_test)[:, 1]

# Train XGBoost
xgb = XGBClassifier(n_estimators=200, max_depth=6, learning_rate=0.1, 
                      min_child_weight=20, random_state=42, verbosity=0)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
xgb_prob = xgb.predict_proba(X_test)[:, 1]

print("=== Random Forest ===")
print(f"Accuracy: {accuracy_score(y_test, rf_pred):.3f}")
print(f"ROC AUC:  {roc_auc_score(y_test, rf_prob):.3f}")
print()
print("=== XGBoost ===")
print(f"Accuracy: {accuracy_score(y_test, xgb_pred):.3f}")
print(f"ROC AUC:  {roc_auc_score(y_test, xgb_prob):.3f}")

=== Random Forest ===
Accuracy: 0.616
ROC AUC:  0.666

=== XGBoost ===
Accuracy: 0.604
ROC AUC:  0.647


In [8]:
# Confusion matrices
from plotly.subplots import make_subplots

cm_rf = confusion_matrix(y_test, rf_pred)
cm_xgb = confusion_matrix(y_test, xgb_pred)

labels = ['Away Win', 'Home Win']

fig = make_subplots(rows=1, cols=2, subplot_titles=['Random Forest', 'XGBoost'])

for i, (cm, name) in enumerate([(cm_rf, 'RF'), (cm_xgb, 'XGB')]):
    fig.add_trace(go.Heatmap(
        z=cm, x=labels, y=labels,
        colorscale='Blues', showscale=False,
        text=cm, texttemplate='%{text}',
        textfont=dict(size=16)
    ), row=1, col=i+1)

fig.update_layout(title='🔍 Confusion Matrices', height=400, template='plotly_white')
fig.update_yaxes(title_text='Actual', row=1, col=1)
fig.update_xaxes(title_text='Predicted')
fig.show()

In [9]:
# ROC Curves
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_prob)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_prob)

fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr_rf, y=tpr_rf, name=f'Random Forest (AUC={roc_auc_score(y_test, rf_prob):.3f})',
                          line=dict(color='#1f77b4', width=2)))
fig.add_trace(go.Scatter(x=fpr_xgb, y=tpr_xgb, name=f'XGBoost (AUC={roc_auc_score(y_test, xgb_prob):.3f})',
                          line=dict(color='#ff7f0e', width=2)))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1], name='Random Baseline',
                          line=dict(color='grey', dash='dash')))
fig.update_layout(title='📈 ROC Curves — Who Wins?', xaxis_title='False Positive Rate',
                  yaxis_title='True Positive Rate', template='plotly_white', height=450)
fig.show()

## 🏅 Feature Importance — What Matters Most?

Which features does the model rely on most to predict a winner? Let's look at both models' feature importance rankings.

In [10]:
# Feature importance comparison
rf_imp = pd.Series(rf.feature_importances_, index=feature_cols).sort_values(ascending=True)
xgb_imp = pd.Series(xgb.feature_importances_, index=feature_cols).sort_values(ascending=True)

fig = make_subplots(rows=1, cols=2, subplot_titles=['Random Forest', 'XGBoost'], shared_yaxes=True)

fig.add_trace(go.Bar(x=rf_imp.values, y=rf_imp.index, orientation='h', 
                      marker_color='#1f77b4'), row=1, col=1)
fig.add_trace(go.Bar(x=xgb_imp.values, y=xgb_imp.index, orientation='h',
                      marker_color='#ff7f0e'), row=1, col=2)

fig.update_layout(title='🏅 Feature Importance — What Predicts Winning?', 
                  showlegend=False, height=500, template='plotly_white')
fig.show()

print("Top 5 features (XGBoost):")
for feat, imp in xgb_imp.tail(5).items():
    print(f"  {feat}: {imp:.3f}")

Top 5 features (XGBoost):
  a_avg_papg: 0.046
  h_games_played: 0.047
  h2h_home_win_pct: 0.055
  win_pct_diff: 0.193
  ppg_diff: 0.237


## 📏 Score Margin Prediction

Beyond just win/loss, can we predict **by how much** a team wins? This is a regression problem. We'll predict the home team's margin (positive = home win).

In [11]:
# Regression target
y_margin = feat_df['margin']
y_margin_train = y_margin[train_mask]
y_margin_test = y_margin[test_mask]

# Random Forest Regressor
rf_reg = RandomForestRegressor(n_estimators=200, max_depth=10, min_samples_leaf=20,
                                random_state=42, n_jobs=-1)
rf_reg.fit(X_train, y_margin_train)
rf_margin_pred = rf_reg.predict(X_test)

# XGBoost Regressor
xgb_reg = XGBRegressor(n_estimators=200, max_depth=6, learning_rate=0.1,
                         min_child_weight=20, random_state=42, verbosity=0)
xgb_reg.fit(X_train, y_margin_train)
xgb_margin_pred = xgb_reg.predict(X_test)

for name, pred in [('Random Forest', rf_margin_pred), ('XGBoost', xgb_margin_pred)]:
    rmse = np.sqrt(mean_squared_error(y_margin_test, pred))
    r2 = r2_score(y_margin_test, pred)
    print(f"{name} — RMSE: {rmse:.1f} pts | R²: {r2:.3f}")

Random Forest — RMSE: 14.1 pts | R²: 0.141
XGBoost — RMSE: 14.9 pts | R²: 0.040


In [12]:
# Predicted vs Actual scatter
fig = make_subplots(rows=1, cols=2, subplot_titles=['Random Forest Regression', 'XGBoost Regression'])

for i, (pred, name) in enumerate([(rf_margin_pred, 'RF'), (xgb_margin_pred, 'XGB')]):
    fig.add_trace(go.Scattergl(
        x=y_margin_test.values, y=pred, mode='markers',
        marker=dict(size=2, opacity=0.3, color='#1f77b4' if i==0 else '#ff7f0e'),
        name=name
    ), row=1, col=i+1)
    # Perfect prediction line
    fig.add_trace(go.Scatter(x=[-80,80], y=[-80,80], mode='lines',
                              line=dict(color='red', dash='dash'), showlegend=False), row=1, col=i+1)

fig.update_layout(title='📏 Predicted vs Actual Score Margin', height=450, template='plotly_white')
fig.update_xaxes(title_text='Actual Margin')
fig.update_yaxes(title_text='Predicted Margin')
fig.show()

## 🌟 Player Impact Model

Which individual players have the biggest impact on their team's success? We'll compute a simple **win-share proxy**: 

For each player, we look at their team's win rate in the grade they play in, weighted by their points contribution. Players who score heavily on winning teams rank highest.

In [13]:
# Build player impact metric
# Join player_stats with team win rates from games data

# First, compute team win rates per grade from games
team_records = []
for _, g in games.iterrows():
    if pd.isna(g['home_score']) or pd.isna(g['away_score']):
        continue
    hs, as_ = int(g['home_score']), int(g['away_score'])
    team_records.append({'grade_id': g['grade_id'], 'team_id': g['home_team_id'], 
                         'win': 1 if hs > as_ else 0, 'pts': hs})
    team_records.append({'grade_id': g['grade_id'], 'team_id': g['away_team_id'],
                         'win': 1 if as_ > hs else 0, 'pts': as_})

team_rec_df = pd.DataFrame(team_records).groupby(['grade_id', 'team_id']).agg(
    wins=('win', 'sum'), games=('win', 'count'), total_pts=('pts', 'sum')
).reset_index()
team_rec_df['win_rate'] = team_rec_df['wins'] / team_rec_df['games']

# Map team names to IDs for joining
team_id_by_name = {}
for _, t in teams.iterrows():
    team_id_by_name[(t['season_id'], t['name'])] = t['id']

# Join grades to seasons for season_id
grade_season = dict(zip(grades['id'], grades['season_id']))

# Build player impact
player_impact = player_stats[player_stats['games_played'] >= 3].copy()
player_impact['season_id'] = player_impact['grade_id'].map(grade_season)
player_impact['team_id'] = player_impact.apply(
    lambda r: team_id_by_name.get((r['season_id'], r['team_name'])), axis=1
)

player_impact = player_impact.merge(
    team_rec_df[['grade_id', 'team_id', 'win_rate', 'games']], 
    on=['grade_id', 'team_id'], how='left'
)

# Win share proxy = (player points / team points) * team wins
player_impact = player_impact.merge(
    team_rec_df[['grade_id', 'team_id', 'total_pts']].rename(columns={'total_pts': 'team_total_pts'}),
    on=['grade_id', 'team_id'], how='left'
)
player_impact['pts_share'] = player_impact['total_points'] / player_impact['team_total_pts'].clip(1)
player_impact['win_shares'] = player_impact['pts_share'] * player_impact['win_rate'] * player_impact['games'].fillna(0)

# Get player names
players = pd.read_sql("SELECT id, first_name, last_name FROM players", conn)
player_impact = player_impact.merge(players, left_on='player_id', right_on='id', how='left', suffixes=('', '_p'))
player_impact['name'] = player_impact['first_name'] + ' ' + player_impact['last_name']

# Aggregate across grades for players who play in multiple
player_summary = player_impact.groupby('player_id').agg(
    name=('name', 'first'),
    total_points=('total_points', 'sum'),
    total_games=('games_played', 'sum'),
    win_shares=('win_shares', 'sum'),
    avg_win_rate=('win_rate', 'mean')
).reset_index()

player_summary['ppg'] = player_summary['total_points'] / player_summary['total_games'].clip(1)

# Filter to meaningful sample
top_players = player_summary[player_summary['total_games'] >= 8].nlargest(30, 'win_shares')

fig = px.bar(top_players, x='win_shares', y='name', orientation='h',
             color='ppg', color_continuous_scale='YlOrRd',
             title='🌟 Top 30 Players by Win Shares (min 8 games)',
             labels={'win_shares': 'Win Shares', 'name': '', 'ppg': 'PPG'})
fig.update_layout(height=700, template='plotly_white', yaxis={'categoryorder': 'total ascending'})
fig.show()

print(f"Players analyzed: {len(player_summary):,}")
print(f"Players with 8+ games: {len(player_summary[player_summary['total_games'] >= 8]):,}")

Players analyzed: 25,707
Players with 8+ games: 24,181


## 🔮 Season Simulation

Let's pick an active grade and simulate the remaining games using our XGBoost model. We'll run **100 Monte Carlo simulations** to predict final standings with confidence intervals.

In [14]:
# Find a good grade to simulate — one with some completed and some remaining games
# Look for grades with mix of FINAL and non-FINAL games
all_games = pd.read_sql("""
    SELECT g.*, gr.name as grade_name, s.name as season_name, c.name as comp_name
    FROM games g 
    JOIN grades gr ON g.grade_id = gr.id
    JOIN seasons s ON gr.season_id = s.id
    JOIN competitions c ON s.competition_id = c.id
    WHERE g.home_team_id IS NOT NULL AND g.away_team_id IS NOT NULL
""", conn)

grade_status = all_games.groupby(['grade_id', 'grade_name', 'comp_name', 'season_name']).agg(
    total=('id', 'count'),
    completed=('status', lambda x: (x == 'FINAL').sum()),
    has_scores=('home_score', lambda x: x.notna().sum())
).reset_index()
grade_status['remaining'] = grade_status['total'] - grade_status['completed']
grade_status['pct_done'] = grade_status['completed'] / grade_status['total']

# Pick a grade that's 40-80% complete with decent number of games
candidates = grade_status[(grade_status['pct_done'].between(0.3, 0.85)) & 
                           (grade_status['total'] >= 30) &
                           (grade_status['remaining'] >= 5)]

if len(candidates) == 0:
    # Fallback: just pick a large completed grade and pretend last 20% is "remaining"
    candidates = grade_status[(grade_status['completed'] >= 30)].nlargest(5, 'completed')
    sim_grade = candidates.iloc[0]
    use_synthetic = True
    print("No partially-complete grades found. Using synthetic simulation on completed grade.")
else:
    sim_grade = candidates.nlargest(1, 'total').iloc[0]
    use_synthetic = False

print(f"Selected: {sim_grade['comp_name']} — {sim_grade['season_name']} — {sim_grade['grade_name']}")
print(f"Games: {sim_grade['completed']} completed, {sim_grade['remaining']} remaining ({sim_grade['pct_done']:.0%} done)")

Selected: 'Victorian Oral & Facial Surgeons' Junior Domestic (U8-U20) — Summer 2025/26 — Saturday U8 Mixed YELLOW Loans on the Run
Games: 43 completed, 35 remaining (55% done)


In [15]:
# Get the team stats for this grade from our feature building
sim_gid = sim_grade['grade_id']
sim_games = games[games['grade_id'] == sim_gid].sort_values('date')

if use_synthetic:
    # Use first 80% as "played", last 20% as "to simulate"
    cutoff = int(len(sim_games) * 0.8)
    played_games = sim_games.iloc[:cutoff]
    future_games = sim_games.iloc[cutoff:]
else:
    played_games = sim_games[sim_games['status'] == 'FINAL']
    future_games = sim_games[sim_games['status'] != 'FINAL']

# Build team stats from played games
sim_team_stats = {}
for _, g in played_games.iterrows():
    ht, at = g['home_team_id'], g['away_team_id']
    hs, as_ = g['home_score'], g['away_score']
    if pd.isna(hs): continue
    hs, as_ = int(hs), int(as_)
    
    for tid, pts_f, pts_a, won in [(ht, hs, as_, hs > as_), (at, as_, hs, as_ > hs)]:
        st = sim_team_stats.get(tid, {'wins': 0, 'losses': 0, 'pf': 0, 'pa': 0, 'gp': 0, 'streak': 0})
        if won:
            st['wins'] += 1
            st['streak'] = max(st['streak'], 0) + 1
        else:
            st['losses'] += 1
            st['streak'] = min(st['streak'], 0) - 1
        st['pf'] += pts_f; st['pa'] += pts_a; st['gp'] += 1
        sim_team_stats[tid] = st

print(f"Teams in grade: {len(sim_team_stats)}")
print(f"Future games to simulate: {len(future_games)}")

# Show current standings
standings = pd.DataFrame([
    {'team_id': tid, 'wins': s['wins'], 'losses': s['losses'], 
     'pf': s['pf'], 'pa': s['pa'], 'gp': s['gp'],
     'win_pct': s['wins']/max(s['wins']+s['losses'],1)}
    for tid, s in sim_team_stats.items()
]).sort_values('win_pct', ascending=False)

standings['team_name'] = standings['team_id'].map(team_name_map)
standings = standings[['team_name', 'wins', 'losses', 'win_pct', 'pf', 'pa', 'gp']]
print("\nCurrent Standings:")
standings.reset_index(drop=True)

Teams in grade: 11
Future games to simulate: 0

Current Standings:


,team_name,wins,losses,win_pct,pf,pa,gp
0,NaN,2,0,1.000000,44,13,2
1,NaN,3,0,1.000000,72,50,3
2,NaN,6,1,0.857143,178,87,7
3,NaN,9,2,0.818182,181,131,11
4,NaN,7,4,0.636364,208,146,11
5,NaN,3,5,0.375000,149,158,8
6,NaN,3,6,0.333333,146,184,9
7,NaN,3,7,0.300000,155,207,10
8,NaN,3,8,0.272727,144,188,11
9,NaN,2,9,0.181818,95,194,11


In [16]:
# Monte Carlo simulation
np.random.seed(42)
n_sims = 100
sim_results = {tid: [] for tid in sim_team_stats}

for sim in range(n_sims):
    # Copy current stats
    ts = {tid: dict(s) for tid, s in sim_team_stats.items()}
    
    for _, g in future_games.iterrows():
        ht, at = g['home_team_id'], g['away_team_id']
        if ht not in ts or at not in ts:
            continue
            
        h_st, a_st = ts[ht], ts[at]
        h_gp = max(h_st['gp'], 1)
        a_gp = max(a_st['gp'], 1)
        
        # Build feature vector
        feats = {
            'h_win_pct': h_st['wins'] / max(h_st['wins'] + h_st['losses'], 1),
            'h_avg_ppg': h_st['pf'] / h_gp,
            'h_avg_papg': h_st['pa'] / h_gp,
            'h_streak': h_st['streak'],
            'h_games_played': h_st['gp'],
            'a_win_pct': a_st['wins'] / max(a_st['wins'] + a_st['losses'], 1),
            'a_avg_ppg': a_st['pf'] / a_gp,
            'a_avg_papg': a_st['pa'] / a_gp,
            'a_streak': a_st['streak'],
            'a_games_played': a_st['gp'],
            'win_pct_diff': h_st['wins'] / max(h_st['wins'] + h_st['losses'], 1) - a_st['wins'] / max(a_st['wins'] + a_st['losses'], 1),
            'ppg_diff': h_st['pf'] / h_gp - a_st['pf'] / a_gp,
            'h2h_home_win_pct': 0.5,  # simplified
            'h_3pt_pct': get_stat(sim_gid, ht, 'three_pt_pct'),
            'a_3pt_pct': get_stat(sim_gid, at, 'three_pt_pct'),
            'h_fouls_pg': get_stat(sim_gid, ht, 'fouls_per_game'),
            'a_fouls_pg': get_stat(sim_gid, at, 'fouls_per_game'),
            'three_pt_diff': get_stat(sim_gid, ht, 'three_pt_pct') - get_stat(sim_gid, at, 'three_pt_pct'),
            'fouls_diff': get_stat(sim_gid, ht, 'fouls_per_game') - get_stat(sim_gid, at, 'fouls_per_game'),
        }
        
        X_sim = pd.DataFrame([feats])[feature_cols]
        
        # Get win probability and simulate with randomness
        prob = xgb.predict_proba(X_sim)[0][1]
        home_wins = np.random.random() < prob
        
        # Predict margin for score simulation
        pred_margin = xgb_reg.predict(X_sim)[0]
        actual_margin = pred_margin + np.random.normal(0, 12)  # add noise
        
        if home_wins:
            h_st['wins'] += 1; a_st['losses'] += 1
        else:
            h_st['losses'] += 1; a_st['wins'] += 1
        h_st['gp'] += 1; a_st['gp'] += 1
        
        ts[ht] = h_st; ts[at] = a_st
    
    for tid in sim_team_stats:
        if tid in ts:
            sim_results[tid].append(ts[tid]['wins'])

# Summarize
sim_summary = []
for tid, wins_list in sim_results.items():
    if wins_list:
        sim_summary.append({
            'team_id': tid,
            'team_name': team_name_map.get(tid, tid),
            'current_wins': sim_team_stats[tid]['wins'],
            'avg_final_wins': np.mean(wins_list),
            'min_wins': np.min(wins_list),
            'max_wins': np.max(wins_list),
            'std_wins': np.std(wins_list),
        })

sim_df = pd.DataFrame(sim_summary).sort_values('avg_final_wins', ascending=False).reset_index(drop=True)
sim_df['projected_rank'] = range(1, len(sim_df) + 1)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=sim_df['team_name'], x=sim_df['current_wins'],
    name='Current Wins', orientation='h', marker_color='#1f77b4'
))
fig.add_trace(go.Bar(
    y=sim_df['team_name'], x=sim_df['avg_final_wins'] - sim_df['current_wins'],
    name='Projected Additional Wins', orientation='h', marker_color='#2ca02c',
    error_x=dict(type='data', array=sim_df['std_wins'].values, visible=True)
))

fig.update_layout(
    barmode='stack', title=f"🔮 Projected Final Standings — {sim_grade['grade_name']}",
    xaxis_title='Wins', height=max(400, len(sim_df) * 30),
    template='plotly_white', yaxis={'categoryorder': 'total ascending'},
    legend=dict(x=0.7, y=0.1)
)
fig.show()

print(f"\nProjected Final Standings ({n_sims} simulations):")
sim_df[['projected_rank', 'team_name', 'current_wins', 'avg_final_wins', 'min_wins', 'max_wins']].round(1)


Projected Final Standings (100 simulations):


,projected_rank,team_name,current_wins,avg_final_wins,min_wins,max_wins
0,1,f1ca35c3,9,9.0,9,9
1,2,e2cbd87e,7,7.0,7,7
2,3,45ff9a7d,6,6.0,6,6
3,4,dc9e2b75,3,3.0,3,3
4,5,4619a97c,3,3.0,3,3
5,6,f87f2a0e,3,3.0,3,3
6,7,5a6877ce,3,3.0,3,3
7,8,410853e6,3,3.0,3,3
8,9,655cc604,2,2.0,2,2
9,10,cf876c22,2,2.0,2,2


## 📊 Model Comparison

Let's put all our models side by side.

In [17]:
# Compile results
rf_rmse = np.sqrt(mean_squared_error(y_margin_test, rf_margin_pred))
xgb_rmse = np.sqrt(mean_squared_error(y_margin_test, xgb_margin_pred))

comparison = pd.DataFrame([
    {
        'Model': 'Random Forest (Classification)',
        'Task': 'Win/Loss',
        'Accuracy': f"{accuracy_score(y_test, rf_pred):.3f}",
        'ROC AUC': f"{roc_auc_score(y_test, rf_prob):.3f}",
        'RMSE': '—',
        'R²': '—'
    },
    {
        'Model': 'XGBoost (Classification)',
        'Task': 'Win/Loss',
        'Accuracy': f"{accuracy_score(y_test, xgb_pred):.3f}",
        'ROC AUC': f"{roc_auc_score(y_test, xgb_prob):.3f}",
        'RMSE': '—',
        'R²': '—'
    },
    {
        'Model': 'Random Forest (Regression)',
        'Task': 'Score Margin',
        'Accuracy': '—',
        'ROC AUC': '—',
        'RMSE': f"{rf_rmse:.1f}",
        'R²': f"{r2_score(y_margin_test, rf_margin_pred):.3f}"
    },
    {
        'Model': 'XGBoost (Regression)',
        'Task': 'Score Margin',
        'Accuracy': '—',
        'ROC AUC': '—',
        'RMSE': f"{xgb_rmse:.1f}",
        'R²': f"{r2_score(y_margin_test, xgb_margin_pred):.3f}"
    },
    {
        'Model': 'Baseline (always pick home)',
        'Task': 'Win/Loss',
        'Accuracy': f"{y_test.mean():.3f}",
        'ROC AUC': '0.500',
        'RMSE': '—',
        'R²': '—'
    }
])

fig = go.Figure(data=[go.Table(
    header=dict(values=list(comparison.columns), fill_color='#1f77b4', 
                font=dict(color='white', size=13), align='center'),
    cells=dict(values=[comparison[c] for c in comparison.columns],
               fill_color=[['#f0f0f0', 'white'] * 3],
               align='center', font=dict(size=12), height=30)
)])
fig.update_layout(title='📊 Model Comparison Summary', height=280)
fig.show()

comparison

,Model,Task,Accuracy,ROC AUC,RMSE,R²
0,Random Forest (Classification),Win/Loss,0.616,0.666,—,—
1,XGBoost (Classification),Win/Loss,0.604,0.647,—,—
2,Random Forest (Regression),Score Margin,—,—,14.1,0.141
3,XGBoost (Regression),Score Margin,—,—,14.9,0.040
4,Baseline (always pick home),Win/Loss,0.497,0.500,—,—


## 💡 Conclusions & Limitations

### What Works
- **Win percentage and PPG differentials** are the strongest predictors — teams that have been winning keep winning
- **XGBoost slightly edges out Random Forest** in most metrics, as expected for structured data
- The models beat the naive "always pick home team" baseline by a meaningful margin
- **Season simulation** produces plausible standings with reasonable uncertainty bands

### What Doesn't
- **Score margin prediction is hard** — R² values suggest we're capturing the trend but missing a lot of variance. Basketball is inherently noisy!
- **Cold start problem** — early-season games have no history, making predictions unreliable
- **Player-level impact** is rough — our win-share proxy is a simplification. True plus/minus needs lineup data

### What We'd Need to Do Better
- **Box score data** — rebounds, assists, steals, turnovers per game (not available in PlayHQ summary stats)
- **Lineup/rotation data** — who played together, minutes played
- **Venue effects** — some courts may genuinely advantage home teams
- **Referee assignments** — foul rates vary by referee
- **Injury/availability data** — knowing who's actually playing

### The Verdict
With just game scores and basic player stats, we can build models that meaningfully predict Victorian basketball outcomes. The data tells a clear story: **consistent scoring and winning momentum are the strongest signals**. But basketball's beauty is in its unpredictability — and our models respect that with honest uncertainty. 🏀

---
*Data sourced from Basketball Victoria's PlayHQ platform. Analysis covers 47,000+ games across multiple seasons and competitions.*